In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [80]:
data = pd.read_csv("./sample11_1.csv", names=['A', 'B', 'C'], delimiter=';').astype(float)
data.sample(10)

,A,B,C
24,0.374313,0.386902,3.830329
16,-0.918474,2.972042,-0.136350
9,2.482038,1.029028,0.294575
3,0.874066,0.176824,3.860822
48,NaN,NaN,0.228617
41,NaN,NaN,1.315156
17,2.869470,1.394746,2.466386
42,NaN,NaN,1.010008
47,NaN,NaN,4.185940
38,NaN,NaN,4.623310


In [72]:
def msb(data):
    groups = [data[g].dropna() for g in data.columns]
    mean = sum(sum(g) for g in groups)/sum(len(g) for g in groups)
    print(mean)
    return sum([len(j) * (j.mean() - mean)**2 for j in groups])/(len(groups)-1)

msb(data)

1.388554651955357


25.29427188059901

In [83]:
import pandas as pd
from scipy.stats import f_oneway
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Разделите данные на группы
grouped_data = [data[col].dropna() for col in data.columns]

# 1) Межгрупповая дисперсия
msb = sum(data[col].var() * len(data[col]) for col in data.columns) / (len(data.columns) - 1)
print(f"1) Межгрупповая дисперсия (MSB): {msb}")

# 2) Среднегрупповая дисперсия
msw = data.values.flatten().var()
print(f"2) Среднегрупповая дисперсия (MSW): {msw}")

# 3) Значение статистики критерия F
f_statistic = msb / msw
print(f"3) Значение статистики критерия F: {f_statistic}")

# 4) Критическое множество K_alpha на уровне значимости 0,03
alpha = 0.03
critical_value = stats.f.ppf(1 - alpha, len(data.columns) - 1, len(data.values.flatten()) - len(data.columns))
print(f"4) Критическое множество K_alpha: {critical_value}")

# 5) Проверка гипотезы о совпадении ожидаемых значений показателей
p_value_anova = f_oneway(*grouped_data)[1]
print(f"5) P-значение ANOVA: {p_value_anova}")
if p_value_anova < 0.05:
    print("   Гипотеза о совпадении ожидаемых значений отвергается.")
else:
    print("   Нет оснований отвергнуть гипотезу о совпадении ожидаемых значений.")

# 6) P-значение и выводы
tukey_results = pairwise_tukeyhsd(data.values.flatten(), range(len(data)))
print(f"6) P-значение для критерия Тьюки: {tukey_results.pvalues[0]}")
if tukey_results.pvalues[0] < 0.05:
    print("   Гипотеза о совпадении средних значений отвергается.")
else:
    print("   Нет оснований отвергнуть гипотезу о совпадении средних значений.")

# 7) 90% доверительные интервалы для ожидаемых значений
X = sm.add_constant(range(len(data)))  # добавляем константу для вычисления интервалов
model = sm.OLS(data.values.flatten(), X).fit()
confidence_intervals = model.conf_int(alpha=0.1)
print("7) 90% доверительные интервалы для ожидаемых значений:")
print(confidence_intervals)


1) Межгрупповая дисперсия (MSB): 200.61629966267398
2) Среднегрупповая дисперсия (MSW): nan
3) Значение статистики критерия F: nan
4) Критическое множество K_alpha: 3.589823483586648
5) P-значение ANOVA: 0.0001395847288308801
   Гипотеза о совпадении ожидаемых значений отвергается.


ValueError: data has 153 elements and groups has 51

In [98]:
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm

# Замените data_arrays на ваши массивы с группами данных
data_arrays = [data[col].dropna().values for col in data.columns]

# 1) Межгрупповая дисперсия
msb = np.var(np.concatenate([len(group) * [group.mean()] for group in data_arrays])).mean()
print(f"1) Межгрупповая дисперсия (MSB): {msb}")

# 2) Среднегрупповая дисперсия
msw = np.var(np.concatenate(data_arrays))
print(f"2) Среднегрупповая дисперсия (MSW): {msw}")

# 3) Значение статистики критерия F
f_statistic = msb / msw
print(f"3) Значение статистики критерия F: {f_statistic}")

# 4) Критическое множество K_alpha на уровне значимости 0,03
alpha = 0.03
k_alpha = len(data_arrays) - 1
critical_value = stats.f.ppf(1 - alpha, k_alpha, np.sum([len(group) for group in data_arrays]) - len(data_arrays))
print(f"4) Критическое множество K_alpha: {critical_value}")

# 5) Проверка гипотезы о совпадении ожидаемых значений показателей
p_value_anova = f_oneway(*data_arrays)[1]
print(f"5) P-значение ANOVA: {p_value_anova}")
if p_value_anova < 0.05:
    print("   Гипотеза о совпадении ожидаемых значений отвергается.")
else:
    print("   Нет оснований отвергнуть гипотезу о совпадении ожидаемых значений.")

# 6) P-значение и выводы
tukey_results = pairwise_tukeyhsd(np.concatenate(data_arrays), np.concatenate([i * np.ones_like(data) for i, data in enumerate(data_arrays)]))
print(f"6) P-значение для критерия Тьюки: {tukey_results.pvalues[0]}")
if tukey_results.pvalues[0] < 0.05:
    print("   Гипотеза о совпадении средних значений отвергается.")
else:
    print("   Нет оснований отвергнуть гипотезу о совпадении средних значений.")

# 7) 90% доверительные интервалы для ожидаемых значений
X = sm.add_constant(np.concatenate([i * np.ones_like(data) for i, data in enumerate(data_arrays)]))  # добавляем константу для вычисления интервалов
model = sm.OLS(np.concatenate(data_arrays), X, missing='drop').fit()
confidence_intervals = model.conf_int(alpha=0.1)
print("7) 90% доверительные интервалы для ожидаемых значений:")
print(confidence_intervals)


1) Межгрупповая дисперсия (MSB): 0.45168342643926823
2) Среднегрупповая дисперсия (MSW): 3.0051131593908975
3) Значение статистики критерия F: 0.15030496439968316
4) Критическое множество K_alpha: 3.6218235345714844
5) P-значение ANOVA: 0.0001395847288308801
   Гипотеза о совпадении ожидаемых значений отвергается.
6) P-значение для критерия Тьюки: 0.13740149298993942
   Нет оснований отвергнуть гипотезу о совпадении средних значений.
7) 90% доверительные интервалы для ожидаемых значений:
[[0.31102893 1.26325889]
 [0.17164766 0.82624868]]


In [95]:
np.var(np.concatenate([len(group) * [group.mean()] for group in data_arrays]))#.mean()

0.45168342643926823